- 스케줄 : 옵티마이져에서 적용
  - 학습률을 에포크별로 학습을 할때 동적으로 조정하는 기법
  - 지수감쇠 : ExponentialDecay
    - 학습률을 지수적으로 감소
  - 구간별 감쇠 : PiecewiseConstanctDecay
    - 구간별로 학습률을 다르게 적용
    - 특정스텝에서 학습률을 크게 변경하고자 할때 유용
  - 다항 감쇠 : PolynormialDecay
  - 코사인 감쇠 : CosinDecay
  - 사용자 지정(커스텀) : 함수를 구현해서 적용

In [5]:
# 예시....

import tensorflow as tf
import math
# 라이브러리
schedule_learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.1,
    staircase=True # 계단식 감소 , False는 연속 감소
)
tf.keras.optimizers.Adam(learning_rate = schedule_learning_rate)

#사용자 정의 함수
def exponential_decay_fn(epoch):
  return 0.01 * 0.1**(epoch/20)
ex_callback =  tf.keras.optimizers.schedules.LearningRateSchedule(exponential_decay_fn)
# model.fit(x_train,y_train, epochs=n_epocs, callbacks=[lr_callback])


# 사용자 정의 함수를 만들어서
# 구간별
def piecewise_constant_decay(epoch):
  if epoch < 5:
    return 0.01
  elif epoch < 15:
    return 0.005
  else:
    return 0.001

lr_callback =  tf.keras.optimizers.schedules.LearningRateSchedule(piecewise_constant_decay)
# model.fit(x_train,y_train, epochs=n_epocs, callbacks=[lr_callback])

- 신경망 학습 순서
  - 입력층 : 피처의 모양을 설정
    - 플렛, 정규화  
  - 은닉층 : 활성화 함수 Relu
    - 배치노말(사용할 경우 은닉층에서 활성화 함수를 사용 안하고)
    - 활성화층(activation)
  - 출력층  
    - 분류 (이진 - sigmoid, 다중 - softmax)
    - 회귀 ( 출력이 1)
           

  - 모델구성이 완료되면
  - 컴파일
    - 옵티마이져, 손실함수 , 평가방법
    - 스케줄
  - 모델 학습
    - 콜벡함수
      - 체크포인트
      - 조기종료
      - 사용자 정의 스케줄
  - 모델 평가
    - evaluate
  - 모델 적용
  - 모델 배포(도커, AWS, 기타등등.)



In [6]:
# CIFAR10
# 신경망
# 콜벡,배치노말,스케줄 등등 가능한 모든 자원을 사용해서 신경망 구성하고 학습
# 전이학습

In [15]:
from tensorflow.keras.applications import ResNet50
base_model = ResNet50()
base_model.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 25,636,712 (97.80 MB)

 Trainable params: 25,583,592 (97.59 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [16]:
import tensorflow as tf
data = tf.keras.datasets.cifar10.load_data()
X = data[0][0]
y = data[0][1]

In [17]:
X.shape, y.shape

((50000, 32, 32, 3), (50000, 1))

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [19]:
input = tf.keras.layers.Input(shape=x_train.shape[1:])
flattern = tf.keras.layers.Flatten()
norm = tf.keras.layers.Normalization()
h1 = tf.keras.layers.Dense(100,activation='relu')
h2 = tf.keras.layers.Dense(100,activation='relu')
output = tf.keras.layers.Dense(10,activation='softmax')
# 조립
x = norm(input)
x = flattern(x)
x = h1(x)
x = h2(x)
output = output(x)
model = tf.keras.Model(input,output)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ normalization_1 (Normalization)      │ (None, 32, 32, 3)           │               7 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 3072)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │         307,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 100)                 │          10,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 318,417 (1.21 MB)

 Trainable params: 318,410 (1.21 MB)

 Non-trainable params: 7 (32.00 B)

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10, batch_size = 64, validation_split=0.2)
model.evaluate(x_test,y_test)

Epoch 1/10
